# 오차역전파법 (backpropagation)

덧셈 노드의 역전파 : 상류의 값을 그대로 흘린다

곱셈 노드의 역전파 : 입력신호가 서로 바뀐채로 흐른다 (순전파시에 입력 신호를 변수에 저장해야함)

#### 곱셈 계층 구현

In [10]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self,x,y):
        self.x = x  #순전파시에 입력신호 저장
        self.y = y
        out = x*y
        
        return out
    
    def backward(self, dout):  #dout : 상류에서 넘어온 미분
        dx = dout * self.y  #곱셈이기 때문에 x와 y가 바뀜
        dy = dout * self.x
        
        return dx,dy

In [11]:
#순전파 구현
apple = 100
apple_num = 2
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_apple_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [12]:
#역전파 구현
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'

#### 덧셈 계층 구현

In [14]:
class AddLayer:
    def __init__(self):
        pass  #초기화가 필요없기 때문에 아무것도 하지말라 라는 뜻
    
    def forward(self,x,y):
        out = x+y
        
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        
        return dx, dy

In [18]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

mul_apple_price = MulLayer()
mul_orange_price = MulLayer()
add_tot_price = AddLayer()
mul_tot_tax_price = MulLayer()

apple_price = mul_apple_price.forward(apple,apple_num)
orange_price = mul_orange_price.forward(orange, orange_num)
tot_price = add_tot_price.forward(apple_price,orange_price)
tot_tax_price = mul_tot_tax_price.forward(tot_price,tax)

print(tot_tax_price)

dtot_tax_price = 1
dtot_price , dtax = mul_tot_tax_price.backward(dtot_tax_price)
dapple_price, dorange_price = add_tot_price.backward(dtot_price)
dapple, dapple_num = mul_apple_price.backward(dapple_price)
dorange, dorange_num = mul_orange_price.backward(dorange_price)

print(dapple, dapple_num, dorange, dorange_num, dtax)

715.0000000000001
2.2 110.00000000000001 3.3000000000000003 165.0 650


## 활성화 함수 계층 구현

#### RELU함수 구현

In [19]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x<=0)  #x가 0보다 크면 false, 작으면 true를 반환한다.
        out = x.copy()  #x를 얕은 copy를 통해 out에 집어넣는다. x를 바꿔도 out은 그대로 유지된다
        out[self.mask] = 0  #mask가 true인 부분(x가 0보다 작은 부분)에 0을 반환한다. out[mask]=0은 mask가 true 값을 갖는 인덱스의 넘파이 값이 모두 0이 되는 것을 의미한다.  
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

#### sigmoid 계층

In [20]:
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out  #역전파시에 사용하기 위해 보관
        
        return out
    
    def backward(self,dout):
        dx = dout * (1.0 - selfout) * self.out  #sigmoid 역전파 간소화 식
        
        return dx

#### affine transformation (아파인 변환) 계층

affine trans. : 신경망의 순전파 시에 수행하는 행렬의 곱 , 보통 노드사이에 스칼라 값이 아닌 넘파이가 흐르기 때문에 많이 사용

In [21]:
class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(x,self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x.t, dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx